In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
df1 = pd.read_csv("Sport car price.csv")
df2 = pd.read_csv("list_cars.csv")
df2.head()
df1.head()

,Car Name,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price
0,"Porsche,911,2022,3,379,331,4,""101,200""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Lamborghini,Huracan,2021,5.2,630,443,2.8,""274,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Ferrari,488 GTB,2022,3.9,661,561,3,""333,750""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Audi,R8,2022,5.2,562,406,3.2,""142,700""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"McLaren,720S,2021,4,710,568,2.7,""298,000""",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Gabungkan Dataset
merged_data = pd.merge(df1, df2, on='Car Name')

In [4]:
# Proses Collaborative Filtering
from sklearn.metrics.pairwise import cosine_similarity

# Proses Collaborative Filtering
df1_matrix = pd.pivot_table(merged_data, values='Price', index='Car Name', columns='Car Model', fill_value=0)
user_similarity = cosine_similarity(df1_matrix)


KeyError: 'Price'